In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dir_targets = '/kaggle/input/playerid-and-targets/results/targets_df_'

In [ ]:
dir_targets_0_100 = dir_targets + '0_100.csv'
targets_df_0_100 = pd.read_csv(dir_targets_0_100)

In [ ]:
targets_df_0_100.head()

날짜 기준 : targets값과 피처값들관의 연관관계 조사하기


In [ ]:
train_df = pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# https://www.kaggle.com/chumajin/eda-of-mlb-for-starter-english-ver
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
targets_date_0 = unpack_json(train_df['nextDayPlayerEngagement'].iloc[0])

In [ ]:
targets_date_0

In [ ]:
rosters_date_0 = unpack_json(train_df['rosters'].iloc[0])

In [ ]:
rosters_date_0

In [ ]:
roster_targets_0 = pd.merge(targets_date_0,rosters_date_0, how='outer',on='playerId')

In [ ]:
roster_targets_0

왜 null 값이 있지???? => 선수가 아닌가??

In [ ]:
player_csv = pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv')

In [ ]:
player_csv.index = player_csv['playerId']
player_csv.drop(['playerId'], axis = 1, inplace = True)

In [ ]:
player_csv

In [ ]:
player_csv.loc[656744]

'642667'이 선수에 없다.???

In [ ]:
player_csv.loc[547989]

선수가 아닌 것으로 추정하고 => targets 이NAN이면 drop

In [ ]:
roster_targets_0.dropna(subset=['target1','target2','target3','target4'])

roster가 NAN인 것은 그날 경기를 안 뛴것.
1. roster NAN값 살려둔 상태에서 corr
2. NaN 제거 후 corr

In [ ]:
corr_NaN_matrix = roster_targets_0.corr()

In [ ]:
corr_NaN_matrix['target1'].sort_values(ascending = False)

status는 없애고 status code는 Active 면 1, 아니면 0을 준후 다시 해보기

In [ ]:
roster_targets_0.drop(['status'], axis = 1, inplace = True)

In [ ]:
roster_targets_0['statusCode'] = roster_targets_0['statusCode'].fillna(0)

In [ ]:
roster_targets_0['statusCode'].replace('A', 1, inplace = True)

In [ ]:
roster_targets_0

In [ ]:
roster_targets_0.corr()

In [ ]:
# roster_targets_0 전체 코드
'''
targets_date_0 = unpack_json(train_df['nextDayPlayerEngagement'].iloc[0])
rosters_date_0 = unpack_json(train_df['rosters'].iloc[0])
roster_targets_0 = pd.merge(targets_date_0,rosters_date_0, how='outer',on='playerId')
player_csv = pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv')
player_csv.index = player_csv['playerId']
player_csv.drop(['playerId'], axis = 1, inplace = True)
roster_targets_0.dropna(subset=['target1','target2','target3','target4'])
roster_targets_0.drop(['status'], axis = 1, inplace = True)
roster_targets_0['statusCode'] = roster_targets_0['statusCode'].fillna(0)
roster_targets_0['statusCode'].replace('A', 1, inplace = True)
roster_targets_0.corr()


=> 전체 날짜로 각자 corr한 후 평균 내기

=> 각 피처 별로 더했다가 한번에 나눠주기

'''


In [ ]:
a = pd.DataFrame([1,2],[3,4])

위와 같은 방식으로 모든 피처 값을 연관관계 찾기
날짜, playerId, TeamId,

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()


def roster_targets_corr(train_df, start, last):
    plus = lambda a, b : a + b
    for i in range(start, last):
        targets_date = unpack_json(train_df['nextDayPlayerEngagement'].iloc[i])
        rosters_date = unpack_json(train_df['rosters'].iloc[i])
        roster_targets = pd.merge(targets_date,rosters_date, how='outer',on='playerId')
        roster_targets.dropna(subset=['target1','target2','target3','target4'], axis = 0, inplace = True)
        roster_targets.drop(['status'], axis = 1, inplace = True)
        roster_targets['statusCode'] = roster_targets['statusCode'].fillna(0)
        roster_targets['statusCode'] = roster_targets['statusCode'].apply(lambda x : 1 if x == 'A' else 0)
        roster_targets_corr = roster_targets.corr()
        if i == start :
            r_t_c = roster_targets_corr
        else :
            r_t_c = r_t_c.combine(roster_targets_corr, plus)
        if i%100 == 0:
            print(i)
    return r_t_c
            

In [ ]:
len(train_df)

In [ ]:
roster_targets_corr = roster_targets_corr(train_df, 0, 1216)

In [ ]:
roster_targets_corr = roster_targets_corr.apply(lambda x : x/1216)
roster_targets_corr

43과 44에서 corr을 했을 때, statusCode가 있기도 하고 없기도 함 => statusCode가 object이다. => 문자열에 대한 처리가 필요함... => 뭐하나 쉬운게 없냐 => Active만 2 나머지는 0으로 대체


### 결론 ###
그나마 target2랑 target4랑 연관관계가 높음, 

roster는 버려도 될듯 함

